1. Config

In [2]:
import koco
import pandas as pd

In [3]:
train_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')

In [9]:
train_dev.keys() # train: 7,896 / dev: 471

dict_keys(['train', 'dev'])

In [20]:
train = pd.DataFrame(train_dev['train'])
dev = pd.DataFrame(train_dev['dev'])

In [14]:
train.head()

,comments,contain_gender_bias,bias,hate,news_title
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate,"""밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"""
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none,"""'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]"""
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate,"""[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)"""
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none,"""'아스달 연대기' 장동건-김옥빈, 들끓는 '욕망커플'→눈물범벅 '칼끝 대립'"""
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”


In [15]:
train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   comments             7896 non-null   object
 1   contain_gender_bias  7896 non-null   bool  
 2   bias                 7896 non-null   object
 3   hate                 7896 non-null   object
 4   news_title           7896 non-null   object
dtypes: bool(1), object(4)
memory usage: 254.6+ KB


In [18]:
dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   comments             471 non-null    object
 1   contain_gender_bias  471 non-null    bool  
 2   bias                 471 non-null    object
 3   hate                 471 non-null    object
 4   news_title           471 non-null    object
dtypes: bool(1), object(4)
memory usage: 15.3+ KB


In [16]:
train['bias'].unique()

array(['others', 'none', 'gender'], dtype=object)

In [17]:
train['hate'].unique()

array(['hate', 'none', 'offensive'], dtype=object)

2. Gemma formatting

In [25]:
def format_instruction(example):
    # bias와 hate 라벨 처리
    bias_label = 0 if example['bias'] == 'none' else 1
    hate_label = 0 if example['hate'] == 'none' else 1

    # 댓글이 악의적인지 여부 결정
    if bias_label == 1 or hate_label == 1:
        malicious_status = "malicious comment"
    else:
        malicious_status = "not malicious comment"
    
    # 댓글을 기준으로 prompt 생성
    prompt_text = (
        "<start_of_turn>user\ncomments: " + example['comments'] + "<end_of_turn>\n"
        "<start_of_turn>model\n"
        "status: " + malicious_status + "\n<end_of_turn>"
    )
    
    return prompt_text

train['prompt'] = train.apply(format_instruction, axis=1)
dev['prompt'] = dev.apply(format_instruction, axis=1)

In [35]:
train = train.drop(['contain_gender_bias', 'bias', 'hate', 'news_title'], axis=1)
dev = dev.drop(['contain_gender_bias', 'bias', 'hate', 'news_title'], axis=1)

In [39]:
train.head()

,comments,prompt
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,<start_of_turn>user\ncomments: (현재 호텔주인 심정) 아1...
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,<start_of_turn>user\ncomments: ....한국적인 미인의 대표...
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",<start_of_turn>user\ncomments: ...못된 넘들...남의 고...
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데","<start_of_turn>user\ncomments: 1,2화 어설펐는데 3,4화..."
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,<start_of_turn>user\ncomments: 1. 사람 얼굴 손톱으로 긁...


3. Model load and Tuning

In [ ]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
model_id = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)